In [ ]:
# Get an Authentication token from https://ngrok.com/
!pip install flask pyngrok neo4j jsonify
!ngrok authtoken '2f34.....'
# ngrok http --domain=scarcely-wired-asp.ngrok-free.app 80
!mkdir -p /root/.ngrok2/
!echo "authtoken: 2f34yq......" > /root/.ngrok2/ngrok.yml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=8251e60e4463218714ea79e3946cdcf001a8efcd6444924fa7fdf0b4076911d0
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
  Created wheel for jsonify: filename=jsonify-0.5-py3-none-any.whl size=1539 sha256=387fc00318725aa95a8d10f7297bee6f3671ed1fb78714ed6e89bd53a7a26238
  Stored in directory: /root/.cache/pip/wheels/4e/6d/b2/9cb590fb5654ff13457f49d4aa5b16469419681613c46a2ec1
Successfully built neo4j jsonify
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from array import array
#from google.colab import output
#output.serve_kernel_port_as_window(5000)
from flask import Flask, request, jsonify
from pyngrok import ngrok
from neo4j import GraphDatabase
import uuid  # For generating unique keys
import traceback
import inspect

problem_relations = [
    ("subgoals", "subgoal", "HAS_SUB_GOAL"),
    ("sub_goals", "subgoal", "HAS_SUB_GOAL"),
    ("tasks", "task", "HAS_TASK"),
    ("solutions", "solution", "HAS_SOLUTION"),
    ("constraints", "constraint", "HAS_CONSTRAINT"),
    ("prompts", "prompt", "HAS_PROMPT"),
    ("tags", "tag", "HAS_TAG"),
    ("resources", "resource", "HAS_RESOURCE"),]

relations_keys = {item[0]: None for item in problem_relations}

myport = 5000
domain = "scarcely-wired-asp.ngrok-free.app"
run_in_thread = False
counter = 0

# Neo4j connection details
uri = "neo4j+s://4fb62461.databases.neo4j.io:7687"
username = "neo4j"
password = "YM3M......"

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Créer une instance de l'application Flask
app = Flask(__name__)

def serialize_neo4j_record(record):
    """Serializes a Neo4j record to a JSON-friendly dictionary."""
    result = {}
    for key in record.keys():
        item = record[key]
        if isinstance(item, list):
            result[key] = [serialize_neo4j_node(node) for node in item]
        else:
            result[key] = serialize_neo4j_node(item)
    return result

def serialize_neo4j_node(node):
    """Serializes a Neo4j Node or Relationship to a dictionary of properties."""
    if node is None:
        return None
    elif hasattr(node, 'properties'):
        # Ensure all properties are serializable
        serialized = {k: serialize_neo4j_node(v) for k, v in node.properties.items()}
        serialized['id'] = node.id  # Add the internal ID of the node or relationship
        return serialized
    elif isinstance(node, (int, float, str)):
        return node  # JSON serializable
    else:
        # Recursively serialize dictionary contents
        return {k: serialize_neo4j_node(v) for k, v in node.items()}

def serialize_neo4j_node(node):
    """Serializes a Neo4j Node, Relationship, or any data to a dictionary."""
    if node is None:
        return None
    elif isinstance(node, list):
        # If it's a list, serialize each item in the list
        return [serialize_neo4j_node(item) for item in node]
    elif hasattr(node, 'properties'):
        # If it has properties, serialize them and add an ID
        serialized = {k: serialize_neo4j_node(v) for k, v in node.properties.items()}
        serialized['id'] = node.id  # Add the internal ID of the node or relationship
        return serialized
    elif isinstance(node, (int, float, str)):
        # If it's a basic type, return it
        return node
    else:
        # Otherwise, assume it's a dictionary and serialize its contents
        return {k: serialize_neo4j_node(v) for k, v in node.items()}

def handle_exception(e, error_number=500):
    # Get the current function name
    current_frame = inspect.currentframe()
    calling_frame = inspect.getouterframes(current_frame, 2)[1]
    function_name = calling_frame.function

    # Capture the traceback information
    traceback_info = traceback.format_exc()  # Gets the full traceback as a string

    # Build the error message
    error_message = {
        "error": str(e),
        "function": function_name,
        "traceback": traceback_info,
    }

    # Print the error information for debugging
    print(f"Error in function '{function_name}': {str(e)}")
    print("Traceback:")
    print(traceback_info)

    # Return a JSON response with the error information
    return jsonify(error_message), error_number

# Route de base
@app.route("/")
def home():
    global counter
    counter += 1
    return "Hello from Colab! Counter:" + str(counter)

@app.route("/problems/<problem_id>", methods=["GET"])
def get_problem(problem_id):
    if problem_id.isdigit():
        filter, parameters = "id(p) = $id", {'id': int(problem_id)}
    else:
        filter, parameters = "p.name =~ $name", {'name': f"(?i).*{problem_id}.*"}

    # Safe parameterization for numeric ID
    query = """
        MATCH (p:Problem)
        WHERE """ + filter + """
        OPTIONAL MATCH (p)-[rel]->(related)
        RETURN id(p) AS problem_id, p AS problem,
                collect(DISTINCT {relation_type: type(rel), related_id: id(related), related_node: related}) AS relatedEntities
    """
    print(f"Query: {query}")

    with driver.session() as session:
        result = session.run(query, parameters=parameters)
        data = [serialize_neo4j_record(record) for record in result]
        if data:
            return jsonify(data)
        else:
            return jsonify({"error": "Problem not found"}), 404

@app.route("/problems", methods=["GET"])
def list_problems():
    # Cypher query to retrieve all problems
    query = """
        MATCH (p:Problem)
        RETURN DISTINCT id(p) as uid, p.name as name, p.context as context, p.goal as goal
        ORDER BY p.name
    """
    with driver.session() as session:
        result = session.run(query)
        data = [{"name": record["name"], "uid": record["uid"], "goal": record["goal"], "context": record["context"]} for record in result]
        if data:
            return jsonify(data)
        else:
            return jsonify({"error": "No problems found"}), 404

def create_or_update_node(tx, parent_key, node_label, relation_label, attributes, relations_keys, parent_node_label='Problem', create=True):
    # Only allow primitive types or arrays for Neo4j properties and do not update key or id
    update_fields = filter_element_fields(attributes)
    node_key = attributes.get("key", str(uuid.uuid4()))  # Generate unique key
    node_label = attributes.get('_node_label', attributes.get('node_label', node_label))

    if attributes.get("_operation", None) == "delete":
        # Delete the node if it exists
        tx.run("""
            MATCH (node:{node_label} {key: $node_key})
            DETACH DELETE node
        """, {"node_key": node_key, "node_label": node_label})
        return node_key

    # Check if the node exists based on the key
    existing_node = tx.run(f"""MATCH (node:{node_label} {{ key: $node_key }}) RETURN node""", {"node_key": node_key}).single()

    if existing_node:
        # Update specific fields only
        tx.run(f"""MATCH (node:{node_label} {{ key: $node_key }}) SET {', '.join([f'node.{k} = ${k}' for k in update_fields.keys()])}""",
            {**update_fields, "node_key": node_key},)
    elif create:
        # Create a new node and link it to the parent
        parent_key = attributes.get('_parent_key', attributes.get('parent_key', parent_key))
        parent_node_label = attributes.get('_parent_node_label', attributes.get('parent_node_label', parent_node_label))
        tx.run(f"""
            MATCH (parent:{parent_node_label} {{ key: $parent_key }})
            CREATE (node:{node_label} {{ key: $node_key, {', '.join([f'{k}: ${k}' for k in update_fields.keys()])} }})
            MERGE (parent)-[:{relation_label}]->(node)
            """,
            {"parent_key": parent_key, "node_key": node_key, **update_fields},
        )
    return node_key  # Return the key used for the node

def filter_element_fields(element, id_excluded=["id"]):
    update_fields = {}
    for k, v in element.items():
        if k not in id_excluded and k not in relations_keys and (isinstance(v, (int, float, bool, str)) or (isinstance(v, list) and not any(isinstance(item, dict) for item in v))):  # Only primitive types or arrays
            update_fields[k] = v
    print(f"UPDATED: {update_fields}")
    return update_fields

def process_nested_elements(tx, parent_key, elements, node_label, relation_label, problem_relations, parent_node_label='Problem', create=True):
    if not isinstance(elements, list): elements = [elements]
    previous_key = None
    for element in elements:
        previous_key = element.get('_previous_key', element.get('previous_key', previous_key))
        cur_node_label = element.get('_node_label', element.get('node_label', node_label))
        current_key = create_or_update_node(tx, parent_key, node_label, relation_label, element, relations_keys, parent_node_label, create=create)
        # Process nested elements recursively
        for key, sub_label, sub_relation in problem_relations:
            if key in element:
                #print(f" process_nested_elements(tx, {current_key}, {element[key]}, {sub_label}, {sub_relation}, {problem_relations})")
                process_nested_elements(tx, current_key, element[key], sub_label, sub_relation, problem_relations, sub_label, create=create)
        # Handle NEXT relationships
        previous_key = element.get('_previous_key', element.get('previous_key', previous_key))
        if previous_key:
            tx.run(
                f"""
                MATCH (prev:{node_label}), (curr:{node_label})
                WHERE prev.key = $previous_key AND curr.key = $current_key
                MERGE (prev)-[:NEXT]->(curr)""",
                {"previous_key": previous_key, "current_key": current_key},
            )
        previous_key = current_key

@app.route("/problems", methods=["POST"])
def create_problem():
    data = request.get_json()
    update_fields = filter_element_fields(data)
    problem_name = data["name"]

    with driver.session() as session:
        if session.run("MATCH (p:Problem {name: $name}) RETURN p", {"name": problem_name}).single():
            return jsonify({"error": "Problem already exists"}), 409

        try:
            with session.begin_transaction() as tx:
              problem_key = data.get("key", str(uuid.uuid4()))
              create_problem_query = """
              MERGE (p:Problem {""" + ', '.join([f'{k}: ${k}' for k in update_fields.keys()]) + """})
              RETURN id(p) AS problem_id, p.key AS problem_key
              """
              #print(create_problem_query) #print(update_fields)
              problem_id_key = tx.run(create_problem_query, {**update_fields}).single()
              problem_id, problem_key = problem_id_key['problem_id'], problem_id_key['problem_key']

              for key, sub_label, sub_relation in problem_relations:
                if data.get(key, None):
                  process_nested_elements(tx, problem_key, data.get(key, []), sub_label, sub_relation, problem_relations, create=True)

              tx.commit()

            return jsonify({
              "message": "Problem created successfully",
              "problem_key": problem_key,
              "problem_id": problem_id,
            }), 201

        except Exception as e:
            return handle_exception(e)

@app.route("/problems/<problem_id>", methods=["PUT"])
def update_problem(problem_id):
    try:
        problem_id_int = int(problem_id)  # Ensure problem ID is an integer
    except ValueError:
        return jsonify({"error": "Invalid problem ID"}), 400

    data = request.get_json()  # Get JSON data from the request
    update_fields = filter_element_fields(data)

    with driver.session() as session:
        try:
            with session.begin_transaction() as tx:
                # Get the key for the problem
                problem_key_query = tx.run("""MATCH (p:Problem) WHERE id(p) = $problem_id RETURN p.key""", {"problem_id": problem_id_int},)
                problem_key = problem_key_query.single().get("p.key", str(uuid.uuid4()))

                # Update the main problem node
                tx.run(f"""MATCH (p:Problem) WHERE id(p) = $problem_id
                    SET {', '.join([f'p.{k}= ${k}' for k in update_fields.keys()])}""",
                    {
                        "problem_id": problem_id_int,
                        **update_fields})

                # Process elements while ensuring proper deletion and re-creation
                for key, sub_label, sub_relation in problem_relations:
                    if data.get(key, None):
                        process_nested_elements(tx, problem_key, data.get(key, []), sub_label, sub_relation, problem_relations, create=False)

                # Commit the transaction after all operations
                tx.commit()

            return jsonify({"message": "Problem updated successfully"}), 200

        except Exception as e:
            return handle_exception(e)

@app.route("/problems/<problem_id>", methods=["DELETE"])
def delete_problem(problem_id):
    try:
        problem_id_int = int(problem_id)
    except ValueError:
        return jsonify({"error": "Invalid problem ID"}), 400

    try:
        with driver.session() as session:
            # Check if the problem exists
            result = session.run("""
                MATCH (p:Problem)
                WHERE id(p) = $id
                RETURN count(p) AS count
            """, {'id': problem_id_int}).single()

            if result["count"] == 0:
                return jsonify({"error": "Problem not found"}), 404

            with session.begin_transaction() as tx:
                # Delete the problem itself
                tx.run("""
                    MATCH (p:Problem)
                    WHERE id(p) = $id
                    DETACH DELETE p
                """, {'id': problem_id_int})

                # Delete any orphaned entities that were related to the problem
                tx.run("""
                    MATCH (e)
                    WHERE (e)<-[:HAS_SUB_GOAL|HAS_TASK|HAS_RESOURCE|HAS_PROMPT|HAS_SOLUTION]-(:Problem)
                    DETACH DELETE e
                """)

                # Delete any node not linked to any Problem
                tx.run("""
                    MATCH (n)
                    WHERE NOT (n)-[:HAS_SUB_GOAL|HAS_TASK|HAS_RESOURCE|HAS_PROMPT|HAS_SOLUTION]-(:Problem)
                    DETACH DELETE n
                """)

                tx.commit()
                return jsonify({"message": "Problem and all its related entities deleted successfully"}), 200

    except Exception as e:
        print(f"error: {e}")
        return jsonify({"error": str(e)}), 500

# Initialiser ngrok lors du démarrage de l'application
def start_app_and_ngrok(app, domain=None, port=5000, run_in_thread=False):
    public_url = ngrok.connect(port, domain=domain, inspect=False).public_url if domain else ngrok.connect(port).public_url  # Assurez-vous que le port correspond à celui utilisé par Flask
    print(f'ngrok tunnel "public_url" accessible on: {public_url} on port: {port}')
    app.config["BASE_URL"] = public_url
    if run_in_thread:
      import threading
      # Start the Flask server in a new thread
      threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
      print("Flask running on a separate THREAD, you have to stop session to stop it")
    else:
      print("Running Flask, you can stop it by stopping cell")
      app.run(port=myport)

if __name__ == "__main__":
    start_app_and_ngrok(app, domain=domain, port=myport, run_in_thread=run_in_thread)

ngrok tunnel "public_url" accessible on: https://scarcely-wired-asp.ngrok-free.app on port: 5000
Running Flask, you can stop it by stopping cell
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:03:33] "GET /problems HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:03:35] "DELETE /problems/734 HTTP/1.1" 200 -


UPDATED: {'key': '05532a0c-51e0-4ef7-b269-9ab5655836f7', 'name': 'NEW Large language model', 'goal': 'Generate a research survey on the given name and context.', 'context': 'A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process.[1] LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.', 'chiko': 'gadaboum', 'title_embedding_1': [0], 'title_embedding_2': [0], 'abstract_embedding_1': [0], 'abstract_embedding_2': [0], 'plan_embedding_1': [0], 'plan_embedding_2': [0], 'embedding1_model': 'text-embedding-ada-002', 'embedding2_model': 'e5-base-v2', 'success': True}
UPDATED: {'description': 'create an extens

INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:03:43] "POST /problems HTTP/1.1" 201 -


Query: 
        MATCH (p:Problem)
        WHERE id(p) = $id
        OPTIONAL MATCH (p)-[rel]->(related)
        RETURN id(p) AS problem_id, p AS problem,
                collect(DISTINCT {relation_type: type(rel), related_id: id(related), related_node: related}) AS relatedEntities
    


...

INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:04:08] "GET /problems/663 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:04:09] "GET /problems HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:15:33] "GET /problems HTTP/1.1" 404 -


UPDATED: {'key': 'problem-05532a0c', 'name': 'NEW Large language model', 'goal': 'Generate a research survey on the given name and context.', 'context': 'A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process.[1] LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.', 'chiko': 'gadaboum', 'title_embedding_1': [0], 'title_embedding_2': [0], 'abstract_embedding_1': [0], 'abstract_embedding_2': [0], 'plan_embedding_1': [0], 'plan_embedding_2': [0], 'embedding1_model': 'text-embedding-ada-002', 'embedding2_model': 'e5-base-v2', 'success': True}
UPDATED: {'key': 'sub_goals-1', 'description': 'create an exte

INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:15:42] "POST /problems HTTP/1.1" 201 -


Query: 
        MATCH (p:Problem)
        WHERE id(p) = $id
        OPTIONAL MATCH (p)-[rel]->(related)
        RETURN id(p) AS problem_id, p AS problem,
                collect(DISTINCT {relation_type: type(rel), related_id: id(related), related_node: related}) AS relatedEntities
    


INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:15:42] "GET /problems/683 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Apr/2024 17:15:43] "GET /problems HTTP/1.1" 200 -
